#**Operasi File.csv**

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle
import matplotlib.pyplot as plt
import io
from nltk.tokenize import TweetTokenizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pymysql

In [4]:
def read_mysql_table(table, host='localhost', user='root', password='rootku', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Convert the result to a Pandas DataFrame
    df = pd.DataFrame(result)
    
    # Assign column names based on the cursor description
    df.columns = [column[0] for column in cursor.description]
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()
    
    return df

In [5]:
table_name = 'input_review'
df = read_mysql_table(table_name)
df.head()

,id_review,nama,tanggal,review
0,29,aku,2023-12-06,bagus sekali
1,30,aku,2023-12-06,bagus sekali


In [6]:
#menyimpan tweet. (tipe data series pandas)
data_reviews = df['review']
data_reviews.head()

0    bagus sekali
1    bagus sekali
Name: review, dtype: object

In [7]:
data_casefolding = data_reviews.str.lower()
data_casefolding.head()

0    bagus sekali
1    bagus sekali
Name: review, dtype: object

##Filtering

Proses dalam menghilangkan karakter-karakter illegal.

Contoh : %, &, >, (,
{, ], 1-9, @uluwatu, http://...

In [8]:
#filtering url -> menggunakan list comprehension dan menggunakan regular expression re.sub mengganti url dengan spasi
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", ulasan) for ulasan in data_casefolding]

#filtering cont -> menggunakan list comprehension, kata cont(continued) dianggap sebagai noise dalam proses analisis data
filtering_cont = [re.sub(r'\(cont\)'," ", Reviews)for Reviews in filtering_url]

#filtering punctuatuion -> menggunakan list comprehension untuk menghapus tanda baca dari setiap ulasan
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', Reviews) for Reviews in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'

#filtering tagger -> menghapus #tagger dan mengganti tagar dengan spasi
filtering_tagger = [re.sub(r'#([^\s]+)', '', Reviews) for Reviews in filtering_punctuation]

#filtering numeric -> menghapus angka dari setiap ulasan dalam dataset dengan spasi
filtering_numeric = [re.sub(r'\d+', ' ', Reviews) for Reviews in filtering_tagger]

data_filtering = pd.Series(filtering_numeric) #pandas series

print (data_filtering[:4])
#print (data_filtering) #jika ingin mencetak semua data_filtering

0    bagus sekali
1    bagus sekali
dtype: object


In [9]:
import nltk
nltk.download('punkt') #model tokenisasi kata
from nltk.tokenize import word_tokenize

#tokenisasi kata pada setiap baris
data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)

[['bagus', 'sekali'], ['bagus', 'sekali']]


[nltk_data] Downloading package punkt to /home/avrians/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
pip install sastrawi

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [11]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [12]:
factory = StopWordRemoverFactory()
ind_stopword = factory.get_stop_words()

#penedefinisian fungsi stopword
def stopword(line):
  temp = list()
  for word in line:
    if(len(word)>3):
      temp.append(word)
  return temp

data_katapenting = [stopword (line) for line in data_tokens]
print(data_katapenting)

[['bagus', 'sekali'], ['bagus', 'sekali']]


In [13]:
#slang word
path_dataslang = open("Data/kamus kata baku-clear.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8',header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

reviews_formal = []
for data in data_tokens:
  data_clean = [replaceSlang(word) for word in data]
  reviews_formal.append(data_clean)
len_reviews_formal = len(reviews_formal)
print(reviews_formal)
len_reviews_formal

[['bagus', 'sekali'], ['bagus', 'sekali']]


2

In [14]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in reviews_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avrians/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Steamming

Proses stemming dilakukan dengan menghapus imbuhan, baik awalan maupun akhiran dari suatu kata untuk mendapatkan kata dasarnya.

In [15]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [16]:
factory = StemmerFactory()
ind_stemmer =factory.create_stemmer()

def stemmer(line):
  temp = list()
  for word in line:
      word = ind_stemmer.stem(word)
      temp.append(word)
  return temp

reviews_dasar = [stemmer (line) for line in reviews_formal]

#cetak 2 baris pertama
print(reviews_dasar[:2])

[['bagus', 'sekali'], ['bagus', 'sekali']]


In [17]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avrians/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/avrians/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/avrians/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
df['review'][1]

'bagus sekali'

In [19]:
#pengembalian kata ke bentuk dasar
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()

def CleanReview(txt):
  txt = re.sub(r'http\S+', ' ', txt) #mengganti semua url dengan spasi
  txt = re.sub(r'[^a-zA-Z]', ' ', txt) #mengganti karakter non-alfabet dengan spasi
  txt = str(txt).lower() #mengubah huruf besar ke kecil
  txt = word_tokenize(txt) #teks menjadi token
  txt = [item for item in txt if item not in data_notstopword] #menghapus kata yg ga dipake (stopword)
  txt = [lemma.lemmatize(word=w, pos='v')for w in txt] #mengembalikan kata ke bentuk dasar
  txt = [i for i in txt if len(i) > 2] #menghapus kata-kata yang terlalu pendek
  txt = ' '.join(txt)
  return txt
df['CleanReview'] = df['review'].apply(CleanReview)

df.head()

,id_review,nama,tanggal,review,CleanReview
0,29,aku,2023-12-06,bagus sekali,bagus sekali
1,30,aku,2023-12-06,bagus sekali,bagus sekali


In [21]:
# Specify the file path of the pickle file
file_path = 'Model/Model_SVM.pkl'

# Read the pickle file
with open(file_path, 'rb') as file:
    data_train = pickle.load(file)

In [22]:
data_train[0]

'sebenernya ini aplikasi biasa saja hanya saja saya mendownload aplikasi ini karena diwajibkan oleh sekolah dan masalahnya aplikasi ini mengambil data pribadi yang tentu saja saya keberatan kalau ingin membantu seseorang ayolah membantu dengan ikhlas dan jagan malah mengambil data pribadi seseorang'

In [ ]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(data_train)
reviews2 = [" ".join(r) for r in reviews_dasar]

In [ ]:
# Contoh menyimpan model dan vectorizer dalam satu file pickle
model_data = {
    'model': svc_model,          # Gantilah dengan model SVM yang sesuai
    'vectorizer': vectorizer     # Gantilah dengan objek vectorizer yang sesuai
}

# Menyimpan model dan vectorizer dalam satu file pickle
with open('Model/tfidf_review.pkl', 'wb') as file:
    pickle.dump(model_data, file)


In [ ]:
import pickle

# Membaca model dan vectorizer dari file pickle
with open('Model/tfidf_review.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
    load_model = loaded_data['model']
    vectorizer = loaded_data['vectorizer']

# Misalnya, reviews2 adalah list dari multi-line string
# pastikan untuk menangani setiap string dengan benar (misalnya, dengan strip())
reviews2 = [review.strip() for review in reviews2]

# Pastikan bahwa reviews2 telah diproses menggunakan vectorizer yang sama
test_vector = vectorizer.transform(reviews2)

# Prediksi dengan model SVM
result = load_model.predict(test_vector)

# Sekarang 'result' berisi hasil prediksi untuk setiap data uji
print(result)


ValueError: X has 3309 features, but SVC is expecting 4 features as input.

In [ ]:
result

In [ ]:
from sklearn.utils.multiclass import unique_labels
unique_labels(result)

In [ ]:
df['label'] = result

In [ ]:
def delete_all_data_from_table(table, host='localhost', user='root', password='rootku', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Delete all data from the specified table
    query = f"DELETE FROM {table}"
    cursor.execute(query)
    
    # Commit the changes
    connection.commit()
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [ ]:
delete_all_data_from_table('input_review')

In [ ]:
def insert_df_into_hasil_model(df, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()

    # Insert each row from the DataFrame into the 'hasil_model' table
    for index, row in df.iterrows():
        query = "INSERT INTO hasil_model (id_review, nama, tanggal, review, label) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, (row['id_review'], row['nama'], row['tanggal'], row['review'], row['label']))

    # Commit the changes
    connection.commit()

    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [ ]:
insert_df_into_hasil_model(df)

In [ ]:
table_name = 'hasil_model'
hasil_df = read_mysql_table(table_name)
hasil_df.to_csv('Data/hasil_model.csv')